In [12]:
import argparse
from importlib import reload
from ultralytics import YOLO
from functions.hough_functions import LineDetector
from functions import other_functions
from collections import defaultdict

import numpy as np
import pandas as pd
import csv
import cv2

#parser = argparse.ArgumentParser(description='Detect objects in an image and save results to a CSV file.')
#parser.add_argument('--image', '-i', type=str, default='inputs/skyline_clip_short.mp4', help='Path to the input image file')
#args = parser.parse_args()

################################################

# BUILD / PULL MODEL

################################################

model = YOLO('models/yolov8n.pt')
#results = model.train(data='coco128.yaml', epochs=3)
#results = model.val()

################################################

# LOAD IMAGE AND DEFINE VARIABLES

################################################

# open video file
video_1 = 'inputs/skyline_clip_short.mp4'
cap = cv2.VideoCapture(video_1)

# Prep the CSV file
csv_file_name = "detected_objects.csv"
with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Frame','Class ID', 'Object Name','Confidence', 'X1', 'Y1', 'X2', 'Y2'])

# Define the codec and create VideoWriter object
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('output_video.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))

# Store the track history
track_history = defaultdict(lambda: [])

# Initialize a list to store detected objects
detected_objects = []
i = 0

In [13]:
################################################

# LOOP THROUGH FRAMES

################################################

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Apply the YOLO object detection model
        results = model(frame)

        # Apply the Hough Line Transform
        detector = LineDetector()
        lines = detector.detect_lines_image(frame, crop=True, output = False)

        # Calculate Distances
        distance, direction = other_functions.putting_it_all_together(lines, 1700, 1000, 9.5)
        
        ################################
        
        ### JOES CODE TO WRITE TO FILE
        
        ################################
        
        detected_objects = results[0].boxes
        csv_file_name = 'detected_objects.csv'
        object_names = results[0].names
        with open(csv_file_name, mode='a', newline='') as file:
            writer = csv.writer(file)
            for box in detected_objects:
                class_id = box.cls[0].item()
                conf = box.conf[0].item()
                cords = box.xyxy[0].tolist()  # formatted as [x1, y1, x2, y2]
                object_name = object_names.get(class_id, 'Unknown')

                # Write the object data to the CSV file
                writer.writerow([i, class_id, object_name, conf, *cords])

                        
        ################################
        
        ### VISUALIZE STUFF
        
        ################################
        
        '''
        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Write the annotated frame to the output video
        out.write(annotated_frame)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)
        '''
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object, writer, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 (no detections), 82.9ms
Speed: 3.9ms preprocess, 82.9ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)


TypeError: Can't convert object to 'str' for 'filename'